In [ ]:
# # import json
# import proteinnetworks
# import numpy as np
# # import os
# import pandas as pd
# # import palettable
# from scipy.cluster import hierarchy
# from collections import defaultdict
# from matplotlib.colors import rgb2hex, colorConverter

import sklearn.metrics
from sklearn import svm


# %matplotlib inline

import subprocess
import os
from IPython.display import Image
from IPython.display import display

import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import glob

from palettable.colorbrewer.qualitative import Dark2_6


# Pymol Viz


This all needs pipelining properly, but now we have the dzs giving the contribution of each residue to the SVM. 

Carl makes a density field $ \rho_B(r) = \sum_{j \in B} \delta_{z_j, B} N \left( r_j, \sigma_j \right)$, i.e a bunch of atom-centred Gaussians of width $\sigma = 0.5 A$. 

I think I'll just colour residues using the b-factor then spectrum it.

In [ ]:
files = glob.glob("*_heme*.npy")

In [ ]:
labels = set(x[:10] for x in files)

In [ ]:
print(labels)

In [ ]:
for label in labels:
    heme_means = np.load(f"{label}_vshemes.npy")*1000  # scale up
    nucleo_means = np.load(f"{label}_vsnucleos.npy")*1000
    # I want to highlight the most similar atoms. So zero out the atoms which have similarity less than the top-20 value.
    top20_threshold = sorted(heme_means)[-20]
    for i, x in enumerate(heme_means):
        if x < top20_threshold:
            heme_means[i] = 0

    top20_threshold = sorted(nucleo_means)[-20]
    for i, x in enumerate(nucleo_means):
        if x < top20_threshold:
            nucleo_means[i] = 0

    
    
    with open(f"../../inputs/protein-heme/{label[:5]}_converted.pdb") as flines:
        data = flines.readlines()
        residueSequenceNumbers = []
        for line in data:
            if line.startswith("ATOM"):
                residueSequenceNumbers.append(int(line[22:26].strip()))
    
    assert len(residueSequenceNumbers) == len(heme_means)
    
    pymolScript = f"load ../../inputs/protein-heme/{label[:5]}.pdb, {label[:5]}\n"
    pymolScript += f"alter {label[:5]}, b=-1\n"

    for resi,b in zip(residueSequenceNumbers, heme_means): # might not work if the residue ids are off
        pymolScript += f"alter resi {resi}, b={b}\n"

    pymolScript += f"""
    #formatting
    bg_color white
    hide all
    #show sticks
    show cartoon
    spectrum b, blue_red
    set opaque_background=0
    set antialias = on
    set line_smooth = 1
    set depth_cue = 1
    set specular = 1
    set surface_quality = 1
    set stick_quality = 15
    set sphere_quality = 2
    set ray_trace_fog = 0.8
    set light = (-0.2,0,-1)

    set ray_shadows, 0
    set surface_mode, 1
    set cartoon_side_chain_helper,on
    zoom
    rebuild
    """
    pymolScript += f"save {label[:5]}.pse \n"
    pymolScript += f"""
    set ray_trace_mode = 1
    png {label[:5]}.png, width=10cm, dpi=300, ray=1
    """

    with open("temp.pml", mode='w') as flines:
        flines.write(pymolScript)

    # Run quietly
    subprocess.run(["pymol", "-c", "temp.pml"])
    os.remove("temp.pml")
    display(Image(f"{label[:5]}.png"))
#     os.remove(f"{pdbRef}.png")

    